# 01 - Parser Development

Notebook pour développer et tester le parser de bulletins PDF.

## Objectifs
1. Charger et valider les données ground truth
2. Explorer la structure des bulletins
3. Développer le parser PDF (quand disponible)

In [5]:
# ruff: noqa: E402
import json
import sys
from pathlib import Path

# Auto-détecter project_root
current = Path.cwd()
while current != current.parent:
    if (current / "pyproject.toml").exists():
        project_root = current
        break
    current = current.parent

sys.path.insert(0, str(project_root))


# Paths
PROJECT_ROOT = project_root
DATA_DIR = PROJECT_ROOT / "data"
GROUND_TRUTH_PATH = DATA_DIR / "ground_truth" / "chiron_ground_truth.json"

print(f"project_root : {PROJECT_ROOT}")
print("Imports chargés")

project_root : c:\Users\Florent\Documents\data_science\chiron
Imports chargés


## 1. Chargement Ground Truth

In [6]:
# Charger le JSON brut
with open(GROUND_TRUTH_PATH, encoding="utf-8") as f:
    data = json.load(f)

print(f"Metadata: {data['metadata']}")
print(f"Nombre d'élèves: {len(data['eleves'])}")

Metadata: {'projet': 'Chiron', 'description': 'Ground truth pour validation des prompts LLM', 'classe': '5ème', 'trimestre': 'T1', 'nb_eleves': 4}
Nombre d'élèves: 4


In [7]:
# Valider avec Pydantic
from src.core.models import GroundTruthDataset

dataset = GroundTruthDataset(**data)
print(f"Dataset validé: {dataset.metadata.classe} - {dataset.metadata.trimestre}")
print(f"Élèves: {[e.eleve_id for e in dataset.eleves]}")

Dataset validé: 5ème - T1
Élèves: ['ELEVE_A', 'ELEVE_B', 'ELEVE_C', 'ELEVE_D']


In [ ]:
# Utiliser la fonction helper
from src.core.data_loader import load_ground_truth_eleves

eleves = load_ground_truth_eleves(GROUND_TRUTH_PATH)
print(f"Chargé {len(eleves)} élèves (avec synthese_ground_truth)")
eleves[0]

## 2. Exploration des données

In [9]:
# Premier élève - structure complète
eleve = dataset.eleves[0]
print(f"Élève: {eleve.eleve_id}")
print(f"Genre: {eleve.genre}")
print(f"Absences: {eleve.absences_demi_journees} demi-journées")
print(f"Engagements: {eleve.engagements}")
print(f"Nb matières: {len(eleve.matieres)}")

Élève: ELEVE_A
Genre: F
Absences: 4 demi-journées
Engagements: ['Déléguée titulaire']
Nb matières: 12


In [10]:
# Matières
import pandas as pd

df_matieres = pd.DataFrame([m.model_dump() for m in eleve.matieres])
df_matieres

,nom,moyenne_eleve,moyenne_classe,appreciation
0,Anglais LV1,15.21,10.83,Bons résultats. ELEVE_A fournit un travail rég...
1,Arts Plastiques,15.00,14.92,"Bon ensemble, bilan satisfaisant, continuez ai..."
2,EPS,8.00,12.79,"Bilan très insuffisant, ELEVE_A n'a pas réussi..."
3,Éducation Musicale,13.00,9.53,C'est très bien quand vous voulez. Votre inves...
4,Espagnol LV2,13.83,10.76,
5,Français,13.21,10.12,Assez bons résultats à l'écrit. La participati...
6,Histoire-Géographie-EMC,14.66,10.77,Bon trimestre ! ELEVE_A est une élève impliqué...
7,Latin et Grec,18.21,17.77,Très bon travail. Élève volontaire et investie.
8,Mathématiques,16.07,11.75,"Très bon trimestre. ELEVE_A a de bons acquis, ..."
9,Physique-Chimie,15.71,12.49,C'est un très bon trimestre. La participation ...


In [11]:
# Synthèse ground truth (ce qu'on veut générer)
print("=" * 60)
print("SYNTHÈSE GROUND TRUTH")
print("=" * 60)
print(eleve.synthese_ground_truth)

SYNTHÈSE GROUND TRUTH
ELEVE_A a obtenu des résultats globalement positifs, entachés par une dispersion importante relevée dans de nombreuses matières. Nous l'invitons donc à se montrer plus régulière dans son travail personnel mais également dans sa concentration en classe afin de réaliser une belle année de 5ème. Nous comptons sur elle.


## 3. Statistiques

In [12]:
# Moyennes par élève
for e in dataset.eleves:
    notes = [m.moyenne_eleve for m in e.matieres if m.moyenne_eleve is not None]
    moy = sum(notes) / len(notes) if notes else 0
    print(f"{e.eleve_id}: moyenne = {moy:.2f}")

ELEVE_A: moyenne = 14.13
ELEVE_B: moyenne = 13.41
ELEVE_C: moyenne = 13.49
ELEVE_D: moyenne = 12.74


In [13]:
# Matières communes
matieres_set = set()
for e in dataset.eleves:
    for m in e.matieres:
        matieres_set.add(m.nom)

print(f"Matières uniques ({len(matieres_set)}):")
for m in sorted(matieres_set):
    print(f"  - {m}")

Matières uniques (13):
  - Anglais LV1
  - Arts Plastiques
  - EPS
  - Espagnol LV2
  - Français
  - Histoire-Géographie-EMC
  - Italien LV2
  - Latin et Grec
  - Mathématiques
  - Physique-Chimie
  - SVT
  - Technologie
  - Éducation Musicale


## 4. Parser PDF (à compléter)

Quand un PDF réel sera disponible, tester ici le parsing.

In [ ]:
# from src.document.bulletin_parser import BulletinParser
#
# parser = BulletinParser()
# pdf_path = DATA_DIR / "raw" / "bulletin_exemple.pdf"
#
# if pdf_path.exists():
#     eleves = parser.parse(pdf_path)
#     print(f"Extrait {len(eleves)} élève(s)")
#     for e in eleves:
#         print(e)
# else:
#     print(f"PDF non trouvé: {pdf_path}")